In [1]:
from zhipuai import ZhipuAI
import copy

prompt = '作为AI地下城主，你的任务是促进一场刺激的《龙与地下城》第五版冒险活动，始终严格遵循规则。协助角色创建或根据玩家偏好生成角色。默认设置是被遗忘的领域，但要根据玩家的愿望进行调整。根据玩家的D&D经验设定难度等级。平衡战斗、谜题和角色扮演，但根据玩家的喜好调整这种混合。如果情节未指定，创造一个引人入胜的故事线。管理经验点数和角色升级。当收到/status命令时，提供当前的经验值、等级、金币和任何条件。在/summary请求时，提供一个战役回顾。在/quests命令下，列出活跃的任务并简要描述。当调用/commands时，详细说明可用的命令。允许玩家选择他们角色的技能、法术、特技和能力，在必要时提供指导。如果玩家在能力检查中失败，通过替代路线引导故事向前发展。不要犹豫在设计机关、挑战性战斗场景和困难决策时设置陷阱。每当需要掷骰子时，告诉玩家该怎么做。你的目标是引导、挑战并适应玩家的行动，以提供一场独特而难忘的冒险。'
instruction = (
    "游戏的最终目标是打败最终BOSS-盖侬并拯救公主，每次回答时都要检测并输出当前距离最终目标的进度占比，数值为0.0～1.0。\n"
    "请按照如下json格式用中文回答:\n"
    "{'result': str, 'status': str, 'goal_percentage': float, 'options': list[str]}\n"
    "其中options最多不超过六个选项。\n"
    "回答仅包含上述json内容，不包含任何其它内容，不使用markdown格式。\n"
)
one_shot = "{\n  'result': '欢迎来到被遗忘的领域！请告诉我你的角色背景、偏好以及经验等级，我将为你定制一场刺激的冒险。',\n  'status': '初始化中，等待角色信息。',\n  'goal_percentage': 0.0,\n  'options': ['创建新角色', '选择预设角色', '调整冒险设置', '查看规则']\n}"
chat_show_keys = ['result', 'status', 'goal_percentage']

print(prompt)
print(instruction)
print(eval(one_shot))

def chat(query:str, history:list=[]):
    client = ZhipuAI(api_key="5c43d4873ee6d67c69c195fc86b5872f.581qTcuFl6ezdTcj") # 填写您自己的APIKey

    messages = copy.copy(history)
    messages.append({"role": "user", "content": query})
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=messages
    )
    
    answer = response.choices[0].message.content
    messages.append({
        "role": response.choices[0].message.role,
        "content": response.choices[0].message.content,
    })
    return messages, answer

def is_convertible_to_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [ ]:
history = [
    {"role": "user", "content": prompt+instruction},
    {"role": "assistant", "content": one_shot},
]
result_json = eval(one_shot)
max_retry = 3


while True:
    # show
    text = ''
    for key in chat_show_keys:
        text += f'{key}: {result_json[key]}\n'
    text += 'options: \n'
    for i, o in enumerate(result_json['options']):
        text += f'\t{i}: {o}\n'
    text += f'\tor: 其它任意决策'
    print(text)

    # finish
    if result_json['goal_percentage'] >= 1.0:
        print("GG~")
        break
    
    # choice
    choice = input('input number or other text: ')
    if is_convertible_to_int(choice) and 0 <= int(choice) < len(result_json['options']):
        choice = result_json['options'][int(choice)]
    print('=' * 100 + '\n' +  '...***' + choice + '***...')
    
    # chat
    retry_num = 0
    while retry_num <= max_retry:
        try:
            history_tmp, answer = chat(choice, history)
            result_json = eval(answer)
            break
        except:
            retry_num += 1
    if retry_num > max_retry:
        print('[error]', history)
        print('[error]', answer)
        print('[error]')
        break
    else:
        history = history_tmp